In [2]:
import cv2
import numpy as np
import math
import glob
import matplotlib.pyplot as plt
from skimage import io ,filters
from mpl_toolkits.mplot3d import Axes3D

In [3]:
# Reading Images

images = []
for filename in glob.glob('set1/*.png'):
    img = cv2.imread(filename) ## cv2.imread reads images in RGB format
    images.append(img)

# img = cv2.imread("set1/a01-132x.png",0) ## cv2.imread reads images in RGB format
# images.append(img)
  
images = np.asarray(images)
plt.imshow(images[0], cmap='gray')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Dell\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-2ee4a68b8630>", line 11, in <module>
    images = np.asarray(images)
  File "C:\Users\Dell\anaconda3\lib\site-packages\numpy\core\_asarray.py", line 85, in asarray
    return array(a, dtype, copy=False, order=order)
MemoryError: Unable to allocate 13.0 GiB for an array with shape (529, 3542, 2479, 3) and data type uint8

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Dell\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'MemoryError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most rece

TypeError: object of type 'NoneType' has no len()

In [ ]:
# Remove salt and pepper noise    
# Remove noise 

for i in range(images.shape[0]):
    median = cv2.medianBlur(images[i],5)
    blur = cv2.GaussianBlur(images[i],(5,5),0)

    
plt.imshow(images[0], cmap='gray')
cv2.imwrite('output.png', images[0])


In [ ]:
# Otsu's Binarization

for i in range(images.shape[0]):
    ret3,images[i] = cv2.threshold(images[i],0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

plt.imshow(images[0], cmap='gray')
cv2.imwrite('output.png', images[0])

In [ ]:
# Remove header and footer

length, width = images[0].shape
up, down, left, right = 0, length - 1, 0, width - 1

minWidthOfLines = width/2
contours,__ = cv2.findContours(images[0], cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

for i in contours:
    x, y, w, h = cv2.boundingRect(i)
    if w < minWidthOfLines:
        continue
    if y < length // 2:
        up = max(up, y + 15)
    else:
        down = min(down, y - 15)
        
left = left + 15
right = right -15
test = images[0][up:down + 1, left:right + 1]
test = np.asarray(test)
plt.imshow(test, cmap='gray')
cv2.imwrite('output.png', test)
                       

In [ ]:
# To crop the image
row, col = test.shape
tolerance = 5

sumOfRows = np.sum(test, axis = 1)
rowIndices = np.where(sumOfRows< (col-tolerance)*255)
up = np.min(rowIndices)
down = np.max(rowIndices)

sumOfColoumns = np.sum(test, axis = 0)
colIndices = np.where(sumOfColoumns< (row-tolerance)*255)
left = np.min(colIndices)
right = np.max(colIndices)


afterCrop = test[up:down + 1, left:right + 1]
afterCrop = np.asarray(afterCrop)
plt.imshow(afterCrop, cmap='gray')
cv2.imwrite('output.png', afterCrop)                      

In [ ]:
# Segmentation of Lines

rowIndicesShifted = np.roll(rowIndices, -1)
rowIndicesShifted = rowIndicesShifted[0]

transitionIndices = np.where(np.abs(rowIndices - rowIndicesShifted) > 10)
transitionIndices = transitionIndices[1]

rowIndices = rowIndices[0]

downIndices= rowIndices[transitionIndices]

transitionIndicesUp = np.insert(transitionIndices,0,-1)
transitionIndicesUp = np.delete(transitionIndicesUp,-1)

upIndices= rowIndices[transitionIndicesUp+1]

In [ ]:
img1 = []
for i in range(transitionIndices.shape[0]):
    img1.append(test[upIndices[i]:downIndices[i] + 1, left:right + 1])
    fig = plt.figure()
    plt.imshow(img1[i], cmap='gray')
    cv2.imwrite('output.png', img1[i]) 
